<a href="https://colab.research.google.com/github/tibayo1/Leukemia-DL-Diagnosis/blob/master/leukemia_dl_diagnosis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Details on the Project

Journal: [Human-level recognition of blast cells in acute myeloid
leukemia with convolutional neural networks](https://www.biorxiv.org/content/10.1101/564039v1.full.pdf)

Datasets:  [RashaSalim Kaggle Datasets of Blood smear images for AML(Acute Myleoid Leukemia) Diagnosis](https://www.kaggle.com/rashasalim/blood-smear-images-for-aml-diagnosis)